<a href="https://colab.research.google.com/github/tafsirnetlifyapp/colab/blob/main/untitled14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Install Tools
# @markdown Klik untuk memproses

!pip install -U faster-whisper moviepy srt
!pip install srt

In [ ]:
# @title 2. Input Media: Transkripsi Murottal Arab (Whisper Medium)
import os, urllib.request, srt
from datetime import timedelta
from faster_whisper import WhisperModel
from tqdm import tqdm
from IPython.display import display, clear_output
import ipywidgets as widgets
import uuid

# --- UI
upload_btn = widgets.FileUpload(accept="audio/*,video/*", multiple=False)
url_input = widgets.Text(placeholder="https://contoh.com/audio.mp3", description="Atau Link:")
transcribe_btn = widgets.Button(description="🎧 Transkripsi", button_style="success")
output_box = widgets.Output()

display(widgets.VBox([
    widgets.Label("🎙 Upload file audio/video atau masukkan link langsung:"),
    upload_btn, url_input, transcribe_btn, output_box
]))

# --- Download & Convert (YouTube or direct MP3/MP4)
def download_and_convert(url):
    print("🌐 Mengunduh dan mengonversi file...")
    filename = f"/tmp/input_{uuid.uuid4().hex}.mp3"
    try:
        urllib.request.urlretrieve(url, filename)
        return filename
    except:
        try:
            import yt_dlp
        except:
            !pip install -q yt-dlp
            import yt_dlp

        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': '/tmp/input.%(ext)s',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'quiet': True
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        return "/tmp/input.mp3"

# --- Transkripsi
def do_transcription(file_path):
    print("⚙️ Memuat model Whisper (medium)...")

    # Deteksi GPU/CPU
    try:
        import torch
        device = "cuda" if torch.cuda.is_available() else "cpu"
    except:
        device = "cpu"

    model_size = "medium"  # 🟢 ganti di sini ke "base" jika mau lebih ringan
    model = WhisperModel(
        model_size,
        device=device,
        compute_type="float16" if device == "cuda" else "int8",
        download_root="/content/.whisper-models"
    )

    print(f"🧠 Transkripsi sedang berlangsung di mode: {device.upper()}")
    segments_generator, _ = model.transcribe(
        file_path,
        language="ar",
        task="transcribe",
        beam_size=5,
        best_of=5,
        vad_filter=False  # nonaktif agar tidak skip ayat awal
    )

    segments = list(tqdm(segments_generator))

    srt_entries = [
        srt.Subtitle(
            index=i + 1,
            start=timedelta(seconds=seg.start),
            end=timedelta(seconds=seg.end),
            content=seg.text.strip()
        )
        for i, seg in enumerate(segments)
    ]

    output_path = "/content/output.srt"
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(srt.compose(srt_entries))

    print("\n✅ Transkripsi selesai!")
    print("📁 File disimpan sebagai `output.srt` di panel File (kiri).")
    print("💡 Klik kanan ➜ Download untuk mengunduh manual.")

# --- Tombol Klik
def on_click_transcribe(b):
    with output_box:
        clear_output()
        if upload_btn.value:
            uploaded_file = next(iter(upload_btn.value.values()))
            ext = uploaded_file['metadata']['name'].split(".")[-1]
            path = f"/tmp/uploaded.{ext}"
            with open(path, "wb") as f:
                f.write(uploaded_file["content"])
            print(f"📥 File diupload: {uploaded_file['metadata']['name']}")
            do_transcription(path)

        elif url_input.value.strip():
            try:
                path = download_and_convert(url_input.value.strip())
                print("📥 File dari link berhasil diunduh.")
                do_transcription(path)
            except Exception as e:
                print(f"❌ Gagal mengambil atau mengonversi file dari link:\n{e}")
        else:
            print("⚠️ Silakan upload file atau isi link terlebih dahulu.")

transcribe_btn.on_click(on_click_transcribe)


In [ ]:
# @title 2B. ALternatif (Whisper Large-v3)
import os, urllib.request, srt
from datetime import timedelta
from faster_whisper import WhisperModel
from tqdm import tqdm
from IPython.display import display, clear_output
import ipywidgets as widgets
import uuid

# --- UI
upload_btn = widgets.FileUpload(accept="audio/*,video/*", multiple=False)
url_input = widgets.Text(placeholder="https://contoh.com/audio.mp3", description="Atau Link:")
transcribe_btn = widgets.Button(description="🎧 Transkripsi", button_style="success")
output_box = widgets.Output()

display(widgets.VBox([
    widgets.Label("🎙 Upload file audio/video atau masukkan link langsung:"),
    upload_btn, url_input, transcribe_btn, output_box
]))

# --- Download & Convert (YouTube or direct MP3/MP4)
def download_and_convert(url):
    print("🌐 Mengunduh dan mengonversi file...")
    filename = f"/tmp/input_{uuid.uuid4().hex}.mp3"
    try:
        urllib.request.urlretrieve(url, filename)
        return filename
    except:
        try:
            import yt_dlp
        except:
            !pip install -q yt-dlp
            import yt_dlp

        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': '/tmp/input.%(ext)s',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'quiet': True
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        return "/tmp/input.mp3"

# --- Transkripsi
def do_transcription(file_path):
    print("⚙️ Memuat model Whisper (large-v3)...")

    # Deteksi apakah GPU tersedia
    try:
        import torch
        device = "cuda" if torch.cuda.is_available() else "cpu"
    except:
        device = "cpu"

    model_size = "large-v3"
    model = WhisperModel(
        model_size,
        device=device,
        compute_type="float16" if device == "cuda" else "int8",
        download_root="/content/.whisper-models"
    )

    print(f"🧠 Transkripsi sedang berlangsung di mode: {device.upper()}")
    segments_generator, _ = model.transcribe(
        file_path,
        language="ar",
        task="transcribe",
        beam_size=10,
        best_of=10,
        vad_filter=False  # Nonaktifkan VAD agar tidak skip awal murottal
    )

    segments = list(tqdm(segments_generator))

    srt_entries = [
        srt.Subtitle(
            index=i + 1,
            start=timedelta(seconds=seg.start),
            end=timedelta(seconds=seg.end),
            content=seg.text.strip()
        )
        for i, seg in enumerate(segments)
    ]

    output_path = "/content/output.srt"
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(srt.compose(srt_entries))

    print("\n✅ Transkripsi selesai!")
    print("📁 File disimpan sebagai `output.srt` di panel File (kiri).")
    print("💡 Klik kanan ➜ Download untuk mengunduh manual.")

# --- Tombol Klik
def on_click_transcribe(b):
    with output_box:
        clear_output()
        if upload_btn.value:
            uploaded_file = next(iter(upload_btn.value.values()))
            ext = uploaded_file['metadata']['name'].split(".")[-1]
            path = f"/tmp/uploaded.{ext}"
            with open(path, "wb") as f:
                f.write(uploaded_file["content"])
            print(f"📥 File diupload: {uploaded_file['metadata']['name']}")
            do_transcription(path)

        elif url_input.value.strip():
            try:
                path = download_and_convert(url_input.value.strip())
                print("📥 File dari link berhasil diunduh.")
                do_transcription(path)
            except Exception as e:
                print(f"❌ Gagal mengambil atau mengonversi file dari link:\n{e}")
        else:
            print("⚠️ Silakan upload file atau isi link terlebih dahulu.")

transcribe_btn.on_click(on_click_transcribe)


In [ ]:
# @title 3. Output Srt
# @markdown Klik untuk memproses
import json, urllib.request, srt, re
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Ambil Quran JSON ---
quran_url = "https://raw.githubusercontent.com/tafsirnetlifyapp/quran/refs/heads/main/quran3.json"
quran_data = json.load(urllib.request.urlopen(quran_url))

# --- Surah dan Jumlah Ayat ---
surah_names = ["Al-Fatihah", "Al-Baqarah", "Ali 'Imran", "An-Nisa'", "Al-Ma'idah", "Al-An'am", "Al-A'raf", "Al-Anfal", "At-Tawbah", "Yunus", "Hud", "Yusuf", "Ar-Ra'd", "Ibrahim", "Al-Hijr", "An-Nahl", "Al-Isra'", "Al-Kahf", "Maryam", "Ta-Ha", "Al-Anbiya'", "Al-Hajj", "Al-Mu'minun", "An-Nur", "Al-Furqan", "Ash-Shu'ara", "An-Naml", "Al-Qasas", "Al-'Ankabut", "Ar-Rum", "Luqman", "As-Sajda", "Al-Ahzab", "Saba'", "Fatir", "Ya-Sin", "As-Saffat", "Sad", "Az-Zumar", "Ghafir", "Fussilat", "Ash-Shura", "Az-Zukhruf", "Ad-Dukhan", "Al-Jathiya", "Al-Ahqaf", "Muhammad", "Al-Fath", "Al-Hujurat", "Qaf", "Adh-Dhariyat", "At-Tur", "An-Najm", "Al-Qamar", "Ar-Rahman", "Al-Waqi'a", "Al-Hadid", "Al-Mujadila", "Al-Hashr", "Al-Mumtahana", "As-Saff", "Al-Jumu'a", "Al-Munafiqun", "At-Taghabun", "At-Talaq", "At-Tahrim", "Al-Mulk", "Al-Qalam", "Al-Haqqa", "Al-Ma'arij", "Nuh", "Al-Jinn", "Al-Muzzammil", "Al-Muddaththir", "Al-Qiyama", "Al-Insan", "Al-Mursalat", "An-Naba'", "An-Nazi'at", "Abasa", "At-Takwir", "Al-Infitar", "Al-Mutaffifin", "Al-Inshiqaq", "Al-Buruj", "At-Tariq", "Al-A'la", "Al-Ghashiyah", "Al-Fajr", "Al-Balad", "Ash-Shams", "Al-Lail", "Ad-Duhaa", "Ash-Sharh", "At-Tin", "Al-'Alaq", "Al-Qadr", "Al-Bayyina", "Az-Zalzalah", "Al-'Adiyat", "Al-Qari'a", "At-Takathur", "Al-Asr", "Al-Humazah", "Al-Fil", "Quraysh", "Al-Ma'un", "Al-Kawthar", "Al-Kafirun", "An-Nasr", "Al-Masad", "Al-Ikhlas", "Al-Falaq", "An-Nas"]
jumlah_ayat = [7,286,200,176,120,165,206,75,129,109,123,111,43,52,99,128,111,110,98,135,112,78,118,64,77,227,93,88,69,60,34,30,73,54,45,83,182,88,75,85,54,53,89,59,37,35,38,29,18,45,60,49,62,55,78,96,29,22,24,13,14,11,11,18,12,12,30,52,52,44,28,28,20,56,40,31,50,40,46,42,29,19,36,25,22,17,19,26,30,20,15,21,11,8,8,19,5,8,8,11,11,8,3,9,5,4,7,3,6,3,5,4,5,6]

# --- Normalisasi & Ekstraksi ---
def normalize_arabic(w): return w.replace("أ","ا").replace("إ","ا").replace("آ","ا")
def extract_words(text): return [normalize_arabic(w) for w in re.sub(r"[^\w\s]|[\d_]", "", text).split()]

# --- UI Builder ---
container = widgets.VBox()
entry_controls = []

def make_entry():
    surah_dd = widgets.Dropdown(
        options=[(f"{i+1:03d} - {surah_names[i]}", i+1) for i in range(114)],
        description="Surah",
        layout=widgets.Layout(width="230px")  # 🔹 dipersempit
    )
    ayat_from = widgets.Dropdown(
        description="Dari",
        layout=widgets.Layout(width="155px")  # 🔹 kecil
    )
    ayat_to = widgets.Dropdown(
        description="Sampai",
        layout=widgets.Layout(width="155px")  # 🔹 kecil
    )
    taawudz_cb = widgets.Checkbox(description="Taawudz")
    basmalah_cb = widgets.Checkbox(description="Basmalah")
    btn_hapus = widgets.Button(
        description="Hapus",
        button_style="danger",
        layout=widgets.Layout(width="80px")  # 🔹 diperkecil
    )

    def update(change):
        idx = change["new"] - 1
        ayat_from.options = list(range(1, jumlah_ayat[idx]+1))
        ayat_to.options = list(range(1, jumlah_ayat[idx]+1))
        ayat_from.value = 1
        ayat_to.value = jumlah_ayat[idx]
    surah_dd.observe(update, names="value")
    update({"new": 98})

    def hapus_click(b):
        container.children = [child for child in container.children if child != box]
    btn_hapus.on_click(hapus_click)

    box = widgets.VBox([
        widgets.HBox([surah_dd, btn_hapus]),
        widgets.HBox([ayat_from, ayat_to]),
        widgets.VBox([taawudz_cb, basmalah_cb])
    ], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='5px 0'))

    return box, (surah_dd, ayat_from, ayat_to, taawudz_cb, basmalah_cb)

def add_entry(_):
    ui, ctrl = make_entry()
    entry_controls.append(ctrl)
    container.children += (ui,)

add_btn = widgets.Button(description="➕ Tambah Entri", button_style="info")
add_btn.on_click(add_entry)

analyze_btn = widgets.Button(description="🔍 Analisis & Generate", button_style="success")
output_box = widgets.Output()
add_entry(None)

# --- Analisis dan Replace SRT ---
def do_analysis(_):
    with output_box:
        clear_output()
        try:
            with open("output.srt", "r", encoding="utf-8") as f:
                srt_entries = list(srt.parse(f.read()))
        except:
            print("❌ File output.srt tidak ditemukan.")
            return

        quran_words = []
        for surah_dd, from_dd, to_dd, taawudz_cb, basmalah_cb in entry_controls:
            if taawudz_cb.value:
                quran_words += ["اعوذ", "بالله", "من", "الشيطان", "الرجيم"]
            if basmalah_cb.value:
                quran_words += ["بسم", "الله", "الرحمن", "الرحيم"]
            for ayat in range(from_dd.value, to_dd.value + 1):
                key = f"{surah_dd.value}:{ayat}"
                if key in quran_data:
                    quran_words += [w.strip() for w in quran_data[key].split("|") if w.strip()]
                else:
                    print(f"⚠️ Ayat tidak ditemukan: {key}")

        print(f"📦 Jumlah kata dari Quran: {len(quran_words)}")

        srt_words = []
        for entry in srt_entries:
            srt_words.extend(extract_words(entry.content))
        print(f"📄 Jumlah kata dari SRT: {len(srt_words)}")

        # --- Pencocokan & Pendeteksian pengulangan ---
        qi = si = 0
        mismatches = []
        repeat_log = []

        while qi < len(quran_words) and si < len(srt_words):
            q = normalize_arabic(quran_words[qi])
            s = srt_words[si]

            if q == s:
                qi += 1
                si += 1
            elif si > 0 and s == srt_words[si - 1]:
                repeat_log.append(f"↻ Duplikat di SRT: '{s}' (SRT pos {si+1})")
                si += 1
            elif qi > 0 and q == normalize_arabic(quran_words[qi - 1]):
                repeat_log.append(f"↻ Duplikat di Quran: '{quran_words[qi]}' (Quran pos {qi+1})")
                qi += 1
            elif si + 1 < len(srt_words) and normalize_arabic(quran_words[qi]) == srt_words[si + 1]:
                repeat_log.append(f"↻ Kata SRT dilewati: '{s}' (SRT pos {si+1})")
                si += 1
            elif qi + 1 < len(quran_words) and normalize_arabic(quran_words[qi + 1]) == s:
                repeat_log.append(f"↻ Kata Quran dilewati: '{quran_words[qi]}' (Quran pos {qi+1})")
                qi += 1
            else:
                mismatches.append((qi, quran_words[qi], srt_words[si]))
                qi += 1
                si += 1

        # --- Catat sisa jika belum habis ---
        while qi < len(quran_words):
            mismatches.append((qi, quran_words[qi], "∅ (kosong di SRT)"))
            qi += 1
        while si < len(srt_words):
            mismatches.append((qi, "∅ (kosong di Quran)", srt_words[si]))
            si += 1

        # --- Tampilkan hasil ---
        if repeat_log:
            print("\n🔁 Pengulangan Kata Terdeteksi:")
            for log in repeat_log:
                print(f"  {log}")
        else:
            print("\n✅ Tidak ada pengulangan kata yang dilewati.")

        if mismatches:
            print("\n❌ Terdapat ketidaksesuaian (maks. 5 ditampilkan):")
            for i, q, s in mismatches[:5]:
                print(f"  ❌ Index {i+1}: Quran='{q}' ≠ SRT='{s}'")
        else:
            print("\n✅ Semua kata cocok!")

        # --- Update isi SRT ---
        result = []
        pointer = 0
        for entry in srt_entries:
            count = len(extract_words(entry.content))
            new_words = quran_words[pointer:pointer + count]
            pointer += count
            entry.content = '\u202B' + ' '.join(new_words) + '\u202C'  # RTL tanpa '|'
            result.append(entry)

        with open("output_updated.srt", "w", encoding="utf-8") as f:
            f.write(srt.compose(result))

        print("\n 📁 File 'output_updated.srt' berhasil disimpan. File disimpan sebagai `output_updated.srt` di panel File ➜ (kiri)")
        print(" 💡 Klik kanan → Download pada `output_updated.srt` untuk mengunduh manual.")



analyze_btn.on_click(do_analysis)

# --- Tampilkan UI
display(widgets.VBox([
    container,
    widgets.HBox([add_btn, analyze_btn]),
    output_box
]))

In [ ]:
# @title 4. Merapihkan SRT (tanda ayat pakai ۝, rapi RTL, bersih total)
import srt
import re

# --- Karakter whitespace tak terlihat yang bisa mengacaukan tampilan ---
invisible_whitespace = ''.join([
    '\u200f', '\u202b', '\u202a', '\u202c', '\u202d', '\u202e',
    '\u00a0', '\u2066', '\u2067', '\u2068', '\u2069', '\ufeff'
])

# --- FUNGSI: cek angka Arab dan ubah ke tanda ayat ۝x ---
def is_arabic_number(word):
    cleaned = re.sub(r'[^\d\u0660-\u0669]', '', word)
    return bool(cleaned) and all(ch in '٠١٢٣٤٥٦٧٨٩0123456789' for ch in cleaned)

def to_ayah_symbol(arabic_digit):
    return f"۝{arabic_digit}"  # ⬅️ simbol ayat klasik

def clean_text(text):
    text = re.sub(f"[{invisible_whitespace}]", "", text)
    return text.rstrip()

# --- BACA FILE ---
with open("output.srt", "r", encoding="utf-8") as f:
    base_srt_entries = list(srt.parse(f.read()))

with open("output_updated.srt", "r", encoding="utf-8") as f:
    updated_entries = list(srt.parse(f.read()))

# --- AMBIL SEMUA TOKEN DARI output_updated.srt ---
all_tokens = []
for entry in updated_entries:
    text = clean_text(entry.content.strip())
    tokens = text.split()
    all_tokens.extend(tokens)

# --- GABUNGKAN ANGKA KE KATA SEBELUMNYA, ANGGKA JADI ۝x tanpa spasi ---
final_words = []
for token in all_tokens:
    if is_arabic_number(token):
        ayah_symbol = to_ayah_symbol(token)
        if final_words:
            final_words[-1] += ayah_symbol  # tempel langsung
        else:
            final_words.append(ayah_symbol)
    else:
        final_words.append(token)

# --- GANTI KATA DI output.srt, ABAIKAN & HAPUS "يا" ---
final_entries = []
word_index = 0

# RTL embedding markers
rtl_start = "\u202B"  # RLE = Right-to-Left Embedding
rtl_end = "\u202C"    # PDF = Pop Directional Formatting

for entry in base_srt_entries:
    original_words = clean_text(entry.content.strip()).split()
    new_words = []

    for word in original_words:
        if word == "يا":
            continue
        elif word_index < len(final_words):
            new_words.append(final_words[word_index])
            word_index += 1
        else:
            new_words.append("⛔ habis kata")

    # Bungkus baris dengan RTL marker
    final_content = rtl_start + clean_text(" ".join(new_words)) + rtl_end

    new_entry = srt.Subtitle(
        index=entry.index,
        start=entry.start,
        end=entry.end,
        content=final_content
    )
    final_entries.append(new_entry)

# --- SIMPAN KE FILE AKHIR ---
with open("output_final.srt", "w", encoding="utf-8") as f:
    f.write(srt.compose(final_entries))

print(f"✅ output_final.srt selesai. Total kata dipakai (tanpa angka & 'يا'): {word_index} dari {len(final_words)}")


In [ ]:
# @title 5. Analisis dan Generate Glyph SRT
import json, urllib.request, srt, re
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import timedelta

# --- Ambil data quran4.json ---
quran4_url = "https://raw.githubusercontent.com/tafsirnetlifyapp/quran/refs/heads/main/quran4.json"
req = urllib.request.Request(quran4_url, headers={'User-Agent': 'Mozilla/5.0'})
with urllib.request.urlopen(req) as response:
    quran4 = json.load(response)

# --- Ambil daftar surah dari API Quran.com ---
surah_url = "https://api.quran.com/api/v4/chapters"
surah_data = json.load(urllib.request.urlopen(
    urllib.request.Request(surah_url, headers={'User-Agent': 'Mozilla/5.0'})
))["chapters"]

# --- Fungsi untuk ambil page ayat tertentu ---
def get_page_number(surah_id, ayat_number):
    try:
        url = f"https://api.quran.com/api/v4/verses/by_key/{surah_id}:{ayat_number}?words=true"
        req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        with urllib.request.urlopen(req) as response:
            data = json.load(response)
            return data["verse"]["page_number"]
    except:
        return None

# --- Buat satu entri UI Surah-Ayat ---
def buat_entry_ui():
    surah_dropdown = widgets.Dropdown(
        options=[(f"{s['id']:03d}. {s['name_arabic']} ({s['name_simple']})", s['id']) for s in surah_data],
        layout=widgets.Layout(width="230px")
    )
    ayat_dari = widgets.Dropdown(layout=widgets.Layout(width="60px"))
    ayat_sampai = widgets.Dropdown(layout=widgets.Layout(width="60px"))
    taawudz = widgets.Checkbox(value=False, description='Taawudz')
    basmalah = widgets.Checkbox(value=True, description='Basmalah')
    hapus_btn = widgets.Button(description="Hapus", button_style="danger", layout=widgets.Layout(width="60px"))
    info_label = widgets.HTML()

    # Update ayat dan halaman
    def update_ayat(change=None):
        sid = surah_dropdown.value
        surah = next(s for s in surah_data if s['id'] == sid)
        options = list(range(1, surah['verses_count'] + 1))
        ayat_dari.options = options
        ayat_sampai.options = options
        ayat_dari.value = 1
        ayat_sampai.value = surah['verses_count']

        # Ambil halaman seluruh ayat
        halaman = set()
        for ayat in range(1, surah['verses_count'] + 1):
            page = get_page_number(sid, ayat)
            if page:
                halaman.add(page)

        if halaman:
            halaman_sorted = sorted(halaman)
            links = [
                f"<a href='https://github.com/dickymiswardi/quran/raw/refs/heads/main/fonts/QCF_P{p:03}.TTF' target='_blank'>QCF_P{p:03}.TTF</a>"
                for p in halaman_sorted
            ]
            info_label.value = f"<b>📄 Halaman Mushaf:</b> {', '.join(map(str, halaman_sorted))}<br>📥 {' – '.join(links)}"
        else:
            info_label.value = "⚠️ Halaman tidak ditemukan."

    update_ayat()
    surah_dropdown.observe(update_ayat, names='value')

    row = widgets.VBox([
        widgets.HBox([
            widgets.Label("Surah", layout=widgets.Layout(width="40px")),
            surah_dropdown,
            hapus_btn,
            widgets.Label("Dari", layout=widgets.Layout(width="30px")),
            ayat_dari,
            widgets.Label("Sampai", layout=widgets.Layout(width="45px")),
            ayat_sampai,
            taawudz,
            basmalah
        ]),
        info_label
    ])

    return {
        "ui": row,
        "widgets": {
            "surah": surah_dropdown,
            "dari": ayat_dari,
            "sampai": ayat_sampai,
            "taawudz": taawudz,
            "basmalah": basmalah,
            "hapus": hapus_btn
        }
    }

# --- Kontainer entri ---
entry_list = []
entry_box = widgets.VBox()

# --- Tambah entri baru ---
def tambah_entri(b=None):
    entry = buat_entry_ui()
    entry_list.append(entry)
    entry_box.children += (entry["ui"],)

    def hapus_entry(btn):
        entry_list.remove(entry)
        entry_box.children = tuple(e["ui"] for e in entry_list)
    entry["widgets"]["hapus"].on_click(hapus_entry)

# --- Tombol dan output ---
tambah_btn = widgets.Button(description="➕ Tambah Entri", button_style="info")
proses_btn = widgets.Button(description="🔍 Analisis & Generate", button_style="success")
output = widgets.Output()
tambah_btn.on_click(tambah_entri)

# --- Fungsi Proses SRT ---
def proses_srt(b):
    output.clear_output()
    with output:
        try:
            with open("output_final.srt", "r", encoding="utf-8") as f:
                original_srt = list(srt.parse(f.read()))
        except FileNotFoundError:
            print("⚠️ File output_final.srt tidak ditemukan.")
            return

        taawudz_text = "أَعُوذُ بِاللَّهِ مِنَ الشَّيْطَانِ الرَّجِيمِ"
        basmalah_text = "بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ"
        glyph_words = []

        for entri in entry_list:
            w = entri["widgets"]
            sid = w["surah"].value
            fa = w["dari"].value
            ta = w["sampai"].value

            if w["taawudz"].value:
                glyph_words += ["اعوذ", "بالله", "من", "الشيطان", "الرجيم"]
            if w["basmalah"].value and sid != 9:
                glyph_words += ["بسم", "الله", "الرحمن", "الرحيم"]

            for ayat in range(fa, ta + 1):
                key = f"{sid}:{ayat}"
                if key in quran4:
                    parts = quran4[key].split("|")
                    for word in parts:
                        word = word.strip()
                        word = re.sub(r"[۝٠-٩()\[\]{}0-9]+", "", word)
                        if word:
                            glyph_words.append(word)

        # Proses SRT
        updated_srt = []
        glyph_index = 0

        for item in original_srt:
            txt = item.content.strip()
            if taawudz_text in txt or basmalah_text in txt:
                new_txt = txt
            else:
                words = re.findall(r'\S+', txt)
                new_words = []
                for _ in words:
                    if glyph_index < len(glyph_words):
                        new_words.append(glyph_words[glyph_index])
                        glyph_index += 1
                    else:
                        new_words.append("⛔ habis kata")
                new_txt = " ".join(new_words)

            updated_srt.append(srt.Subtitle(
                index=item.index,
                start=item.start,
                end=item.end,
                content=new_txt
            ))

        with open("output_glyph.srt", "w", encoding="utf-8") as f:
            f.write(srt.compose(updated_srt))
        print("✅ Selesai. File disimpan sebagai: output_glyph.srt")

# --- Tampilkan UI awal ---
tambah_entri()
display(entry_box, widgets.HBox([tambah_btn, proses_btn]), output)
proses_btn.on_click(proses_srt)
